In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.2MB 6.0MB/s 
     |████████████████████████████████| 3.3MB 22.3MB/s 
     |████████████████████████████████| 870kB 47.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=434400cd2bff5305702240133dafaf3dc93ce07fdf8efa662759b96e649972b7
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import itertools

# from transformers import DistilBertTokenizerFast
# from transformers import TFDistilBertModel, DistilBertConfig

from transformers import BertTokenizer, TFBertForMaskedLM




In [ ]:
lav_path = '/content/gdrive/MyDrive/W266Project_Lav_Shalz/train-balanced-sarcasm.csv'
shalz_path = '/content/gdrive/MyDrive/Colab Notebooks/train-balanced-sarcasm.csv'

Data Cleaning

In [ ]:
# df = pd.read_csv(lav_path)
df = pd.read_csv(shalz_path)

In [ ]:
df.shape

(1010826, 10)

In [ ]:
df.isna().sum()

label              0
comment           53
author             0
subreddit          0
score              0
ups                0
downs              0
date               0
created_utc        0
parent_comment     0
dtype: int64

In [ ]:
df = df[df['comment'].notna()]
df.isna().sum()

label             0
comment           0
author            0
subreddit         0
score             0
ups               0
downs             0
date              0
created_utc       0
parent_comment    0
dtype: int64

In [ ]:
df["parent_comment"][0]

"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well."

In [ ]:
df['subreddit'].value_counts()

# pick top 20

AskReddit               65674
politics                39493
worldnews               26376
leagueoflegends         21034
pcmasterrace            18987
                        ...  
worldofnintendoswap         1
raw                         1
BrotherhoodOfBolland        1
shittyactionfigures         1
spotthevegan                1
Name: subreddit, Length: 14876, dtype: int64

In [ ]:
df[df["subreddit"] == "AskReddit"]

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
5,0,"I don't pay attention to her, but as long as s...",only7inches,AskReddit,0,0,0,2016-09,2016-09-02 10:35:08,do you find ariana grande sexy ?
6,0,Trick or treating in general is just weird...,only7inches,AskReddit,1,-1,-1,2016-10,2016-10-23 21:43:03,What's your weird or unsettling Trick or Treat...
15,0,what the fuck,Pishwi,AskReddit,22,-1,-1,2016-11,2016-11-04 20:10:33,"Star Wars, easy. I'm not that bothered about I..."
27,0,This would make me cry.,neutralneutrals,AskReddit,1,-1,-1,2016-11,2016-11-07 12:03:16,"""You are like the end piece of bread in a loaf..."
37,0,My stuffed animal I've had since I was born.,kn1820,AskReddit,1,-1,-1,2016-10,2016-10-11 11:17:42,Your house is burning down and you only have t...
...,...,...,...,...,...,...,...,...,...,...
1010778,1,...I was waiting for the,inanimal,AskReddit,5,5,0,2009-09,2009-09-24 21:00:04,are you kidding the only reason you don't like...
1010779,0,Hi,mercurysquad,AskReddit,3,3,0,2009-03,2009-03-14 14:22:17,I don't really need a notifier. The envelope i...
1010786,1,"He meant Freedom and Liberties for him, not fo...",tbutters,AskReddit,1,1,0,2009-10,2009-10-18 17:44:42,"What about your second point... ""Freedom and l..."
1010793,1,How admirable.,ryksych,AskReddit,4,4,0,2009-07,2009-07-07 04:40:23,My Catholic high school didn't let a girl part...


In [ ]:
df.iloc[6]

label                                                             0
comment               Trick or treating in general is just weird...
author                                                  only7inches
subreddit                                                 AskReddit
score                                                             1
ups                                                              -1
downs                                                            -1
date                                                        2016-10
created_utc                                     2016-10-23 21:43:03
parent_comment    What's your weird or unsettling Trick or Treat...
Name: 6, dtype: object

Masked Models

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForMaskedLM.from_pretrained('bert-base-cased')

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [ ]:
# test it out
inputs = tokenizer("The capital of France is [MASK].", return_tensors="tf")
mask_position = np.where(inputs['input_ids'] == 103)

In [ ]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


## Testing

In [ ]:
mask_position

(array([0]), array([6]))

In [ ]:
np.argmax(model(inputs)[0][0, mask_position[1][0]])


2123

In [ ]:
tokenizer.convert_ids_to_tokens(2123)


'Paris'

In [ ]:
# # test out with one example

# test_comment = df.iloc[6, 1]
# test_parent_comment = df.iloc[6, 9]
# test_label = df.iloc[6, 0]
# test_phrase = " I am [MASK]!"

# # # can't put three things in because it expects a pair
# tokenizer([[test_parent_comment, test_comment, test_phrase]])

In [ ]:
#test_phrase = " I had a [MASK] experience."
# test_phrase = " The Reddit comment was [MASK]."
# test_phrase = " The previous comment was [MASK]."
# test_phrase = " The previous sentence was [MASK]."

test_phrase1 = " I am [MASK]!"
test_phrase2 = " I am being [MASK]!"
test_phrase3 = " OP is [MASK]!"
test_phrase3 = " OP is [MASK]."

test_phrases = [test_phrase1, test_phrase2, test_phrase3, test_phrase3]

# word1 = ['joking', 'ironic', 'satirical', 'exaggerating', 'sarcastic']
# word2 = ['serious', 'honest', 'blunt', 'transparent', 'unsarcastic']

word1 = ['joking', 'ironic', 'satirical', 'sarcastic' ,'kidding']
word2 = ['serious', 'honest']

#pairs =  list(itertools.product(word1, word2))
pairs = (('joking', 'serious'), ('ironic', 'serious'), ('satirical', 'honest'), ('kidding', 'serious'))

#for phrase in test_phrases:
phrase = test_phrase1
for pair in pairs:
  # phrase_list = [phrase]*20
  comments_pet = [comment + phrase for comment in comments]
  combined = [list(i) for i in zip(parent_comments, comments_pet)]

  # # for a single example
  # comments_pet = comments[1] + phrase
  # combined = [[parent_comments[1], comments_pet]]
  

  inputs = tokenizer(combined, padding=True, return_tensors="tf", max_length=512)

  mask_positions = np.where(inputs['input_ids'] == 103)

  out = model(inputs)

  pair_tokens = tokenizer.convert_tokens_to_ids(pair)

  for example_nr, (context, example) in enumerate(combined):
      print('"' + context + '"')
      print('"' + example + " ")
      print("Label is: ", labels[example_nr])
      print('Logits:')

      print('\t' + pair[0] + ': ', out[0][example_nr, mask_positions[1][example_nr]][pair_tokens[0]].numpy())
      print('\t' + pair[1] + ': ', out[0][example_nr, mask_positions[1][example_nr]][pair_tokens[1]].numpy())
      print()

"I KNOW THEY DIDNT TAKE OUT THE CHICKEN NOISE WHEN YOU GO TO PRESTIGE BUT THEN BACK OUT"
"Your title suggests they didn't remove it but your joke about it now being unplayable suggests they did remove it, which is it? I am [MASK]! 
Label is:  0
Logits:
	joking:  7.446527
	serious:  8.699382

"What is your go to home remedy for bad sinuses and a sore throat?"
"Lemon, honey and ginger I am [MASK]! 
Label is:  0
Logits:
	joking:  3.8863106
	serious:  3.986889

"tipo cose considerate come i capisaldi del rap italiano, se ce ne sono"
"Fedez I am [MASK]! 
Label is:  1
Logits:
	joking:  2.9425082
	serious:  5.0033016

"No mention of the dark lord nestle."
"Oh there not that bad, it's not like they use child slavery to facilitate these mass extinctions. I am [MASK]! 
Label is:  1
Logits:
	joking:  5.6756773
	serious:  7.5164146

"Exakt"
"Tja so viel Pech muss man erstmal haben xD I am [MASK]! 
Label is:  0
Logits:
	joking:  4.3441653
	serious:  5.6929

"Tyson Fury on Twitter: "Don't be fooled 

In [ ]:
print('cols:', len(combined))
print('rows:', len(combined[0]))

cols: 20
rows: 3


## Testing on 20 examples

In [ ]:
# select a subset of the data
s0 = df.label[df.label.eq(0)].sample(10).index
s1 = df.label[df.label.eq(1)].sample(10).index 
df_subset = df.loc[s0.union(s1)].reset_index()
df_subset.head()

,index,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,4133,0,M E T A,Meowfia,AskReddit,1,-1,-1,2016-11,2016-11-14 17:50:58,Then you send her the entirety of Moby Dick via text message
1,139031,1,"But as long as they didn't succeed in killing anyone nobody would be able to prove they're NOT Skrulls, so I think they'd still be in the clear.",Thebxrabbit,politics,1,-1,-1,2016-10,2016-10-30 02:48:56,"Disagree. That's not an excuse. I mean, let's say I tell you that all Cuban people are secretly Skrulls so you drive to Little Havana and attack a bunch of them with a golf club. Obviously I'm not a good person and I should stop spreading misinformation, but that doesn't let you off the hook for your crazed golf-club rampage."
2,148571,1,"damn right, he was out greatest president!",MrKayMkay,worldnews,1,1,0,2016-09,2016-09-15 11:08:32,Australians petition to put Steve Irwin on their money.
3,162923,0,"but then you would b**e** stuck on ps4 for another year or two, so that cant b**e** smarter.",McAwesome03,pcmasterrace,1,1,0,2016-09,2016-09-13 04:51:08,"Yeah but I don't have enough money to spend $1k on a rig and keep my PS, although I'd love to. I guess I could wait even longer and just keep putting money aside and then buy one in a year or two :p, that might be the smarter option."
4,166167,0,Bob Saget for Greg Giraldo,Snoochey,AskReddit,2,-1,-1,2016-10,2016-10-24 09:07:44,"If you could swap a living celebrity for a dead one, who would you choose?"


In [ ]:
comments = list(df_subset['comment'].values)
parent_comments = list(df_subset['parent_comment'].values)
labels = list(df_subset['label'].values)

print(comments)
print(parent_comments)
print(labels)

['M E T A', "But as long as they didn't succeed in killing anyone nobody would be able to prove they're NOT Skrulls, so I think they'd still be in the clear.", 'damn right, he was out greatest president!', 'but then you would b**e** stuck on ps4 for another year or two, so that cant b**e** smarter.', 'Bob Saget for Greg Giraldo', 'I already stated why.', 'smh but draymond allowed to call the refs pussies', '"finishing *touch*" I see what you did there...', 'God forbid we have brown people outnumber us!', 'oh yeah, the sexist tone', 'Snek is doing an anger over his name.', '"unpredictability" "bigger picture" "feels like war" ye, back to 2013, m8.', 'what grade level is this?', "The Guardian doesn't censor, they shut down people who commit hate crimes.", "I hear that song on ESPN every single time it's on.", "Are you sure it's not because YOU'RE A SEXIST?", 'Ah, what a joke!', '*Justice boner intensifies*', 'ITS ALMOST LIKE ITS /R/GAMING IN HERE.', 'We totally believe you.']
['Then you 

In [ ]:
# create a dataframe which has the logit and what phrase/word information so it's easier to analyze
# and see which phrases and words we should use

# columns: parent comment, comment, label, test phrase, word1, word2, word1 logit, word2 logit

test_phrase1 = " I am [MASK]!"
test_phrase2 = " I am being [MASK]!"
test_phrase3 = " OP is [MASK]!"
test_phrase4 = " OP is [MASK]."

test_phrases = [test_phrase1, test_phrase2, test_phrase3, test_phrase4]

# test_phrases = [test_phrase1]


word1 = ['joking', 'ironic', 'satirical', 'exaggerating', 'sarcastic', 'kidding']
word2 = ['serious', 'honest', 'blunt', 'transparent', 'unsarcastic']

# word1 = ['joking', 'ironic', 'satirical', 'sarcastic' ,'kidding']
# word2 = ['serious', 'honest']

# word1 = ['joking', 'ironic']
# word2 = ['serious']

# pairs = (('joking', 'serious'), ('ironic', 'serious'), ('satirical', 'honest'), ('kidding', 'serious'))


# pairs = [list(i) for i in zip(word1, word2)]
pairs = list(itertools.product(word1, word2))

df_parent_list = []
df_comment_list = []
df_label_list = []
df_testphrase_list = []
df_word1_list = []
df_word2_list = []
df_logit1_list = []
df_logit2_list = []

# test_comment = df.iloc[6, 1]
# test_parent_comment = df.iloc[6, 9]
# test_label = df.iloc[6, 0]
# test_phrase = " I am [MASK]!"

# phrase = test_phrase1

for phrase in test_phrases:
  for pair in pairs:
  # for w1 in word1:
  #   for w2 in word2:

    comments_pet = [comment + phrase for comment in comments]
    combined = [list(i) for i in zip(parent_comments, comments_pet)]

    inputs = tokenizer(combined, padding=True, return_tensors="tf", max_length=40)

    mask_positions = np.where(inputs['input_ids'] == 103)

    out = model(inputs)

    pair_tokens = tokenizer.convert_tokens_to_ids(pair)


    for example_nr, (context, example) in enumerate(combined):
        df_parent_list.append(context)
        df_comment_list.append(example)
        df_label_list.append(labels[example_nr])
        df_testphrase_list.append(phrase)
        df_word1_list.append(pair[0])
        df_word2_list.append(pair[1])
        df_logit1_list.append(out[0][example_nr, mask_positions[1][example_nr]][pair_tokens[0]].numpy())
        df_logit2_list.append(out[0][example_nr, mask_positions[1][example_nr]][pair_tokens[1]].numpy())

        # print('"' + context + '"')
        # print('"' + example + " ")
        # print("Label is: ", labels[example_nr])
        # print("Phrase", phrase)
        # print('Logits:')

        # print('\t' + pair[0] + ': ', out[0][example_nr, mask_positions[1][example_nr]][pair_tokens[0]].numpy())
        # print('\t' + pair[1] + ': ', out[0][example_nr, mask_positions[1][example_nr]][pair_tokens[1]].numpy())
        # print()


df_dict = {"parent_comment" : df_parent_list, 
            "comment" : df_comment_list, 
            "test_phrase" : df_testphrase_list, 
            "label" : df_label_list, 
            "word1" : df_word1_list, 
            "word2" : df_word2_list, 
            "word1_logit" : df_logit1_list, 
            "word2_logit" : df_logit2_list} 

cloze_df = pd.DataFrame(df_dict)




In [ ]:
print(len(comments))
print(len(test_phrases))
print(len(word1))
print(len(word2))
print(len(comments) * len(test_phrases) * len(word1) * len(word2))
print(cloze_df.shape)

20
4
6
5
2400
(2400, 8)


In [ ]:
pairs

[('joking', 'serious'),
 ('joking', 'honest'),
 ('joking', 'blunt'),
 ('joking', 'transparent'),
 ('joking', 'unsarcastic'),
 ('ironic', 'serious'),
 ('ironic', 'honest'),
 ('ironic', 'blunt'),
 ('ironic', 'transparent'),
 ('ironic', 'unsarcastic'),
 ('satirical', 'serious'),
 ('satirical', 'honest'),
 ('satirical', 'blunt'),
 ('satirical', 'transparent'),
 ('satirical', 'unsarcastic'),
 ('exaggerating', 'serious'),
 ('exaggerating', 'honest'),
 ('exaggerating', 'blunt'),
 ('exaggerating', 'transparent'),
 ('exaggerating', 'unsarcastic'),
 ('sarcastic', 'serious'),
 ('sarcastic', 'honest'),
 ('sarcastic', 'blunt'),
 ('sarcastic', 'transparent'),
 ('sarcastic', 'unsarcastic'),
 ('kidding', 'serious'),
 ('kidding', 'honest'),
 ('kidding', 'blunt'),
 ('kidding', 'transparent'),
 ('kidding', 'unsarcastic')]

In [ ]:
comments 

['M E T A',
 "But as long as they didn't succeed in killing anyone nobody would be able to prove they're NOT Skrulls, so I think they'd still be in the clear.",
 'damn right, he was out greatest president!',
 'but then you would b**e** stuck on ps4 for another year or two, so that cant b**e** smarter.',
 'Bob Saget for Greg Giraldo',
 'I already stated why.',
 'smh but draymond allowed to call the refs pussies',
 '"finishing *touch*" I see what you did there...',
 'God forbid we have brown people outnumber us!',
 'oh yeah, the sexist tone',
 'Snek is doing an anger over his name.',
 '"unpredictability" "bigger picture" "feels like war" ye, back to 2013, m8.',
 'what grade level is this?',
 "The Guardian doesn't censor, they shut down people who commit hate crimes.",
 "I hear that song on ESPN every single time it's on.",
 "Are you sure it's not because YOU'RE A SEXIST?",
 'Ah, what a joke!',
 '*Justice boner intensifies*',
 'ITS ALMOST LIKE ITS /R/GAMING IN HERE.',
 'We totally believe

In [ ]:
cloze_df

,parent_comment,comment,test_phrase,label,word1,word2,word1_logit,word2_logit
0,Then you send her the entirety of Moby Dick via text message,M E T A I am [MASK]!,I am [MASK]!,0,joking,serious,4.841153,6.183966
1,"Disagree. That's not an excuse. I mean, let's say I tell you that all Cuban people are secretly Skrulls so you drive to Little Havana and attack a bunch of them with a golf club. Obviously I'm not a good person and I should stop spreading misinformation, but that doesn't let you off the hook for your crazed golf-club rampage.","But as long as they didn't succeed in killing anyone nobody would be able to prove they're NOT Skrulls, so I think they'd still be in the clear. I am [MASK]!",I am [MASK]!,1,joking,serious,5.038033,6.425537
2,Australians petition to put Steve Irwin on their money.,"damn right, he was out greatest president! I am [MASK]!",I am [MASK]!,1,joking,serious,5.006810,6.432206
3,"Yeah but I don't have enough money to spend $1k on a rig and keep my PS, although I'd love to. I guess I could wait even longer and just keep putting money aside and then buy one in a year or two :p, that might be the smarter option.","but then you would b**e** stuck on ps4 for another year or two, so that cant b**e** smarter. I am [MASK]!",I am [MASK]!,0,joking,serious,4.316962,5.850067
4,"If you could swap a living celebrity for a dead one, who would you choose?",Bob Saget for Greg Giraldo I am [MASK]!,I am [MASK]!,0,joking,serious,5.143985,6.263018
...,...,...,...,...,...,...,...,...
2395,I had Alg2 with her last year.,Are you sure it's not because YOU'RE A SEXIST? OP is [MASK].,OP is [MASK].,1,kidding,unsarcastic,3.088112,1.234897
2396,Conquerer*,"Ah, what a joke! OP is [MASK].",OP is [MASK].,1,kidding,unsarcastic,2.236543,1.045647
2397,"All you assholes who think you're better than anyone. So what if a fat person gets fast food, its their life. Guess what they pay taxes too, so the whole drain on the health care line is BS. You judge smokers? Fuck you, like you've never had a bad habit. Alcohol kills FAR more people a year than smoking. So if you're on your high horse, fuck you. I've judged you guilty of hypocrisy and want nothing to do with you. Other than that, cheers have a great fucking day :)",*Justice boner intensifies* OP is [MASK].,OP is [MASK].,0,kidding,unsarcastic,-0.557331,1.914264
2398,omg! now you are complaining about him complaining about complaining about complaining! this is too much.,ITS ALMOST LIKE ITS /R/GAMING IN HERE. OP is [MASK].,OP is [MASK].,1,kidding,unsarcastic,1.185381,2.469634


In [ ]:
# cloze_df[cloze_df["comment"] == "Foxnews is the most accurate reporting service next to the bible. OP is [MASK]."]

In [ ]:
# see which word combo is predicting correctly
# create a new column which is 1 if it's predicted correctly (logit is higher for the right word) and 0 if it's not

predicted_labels = []
for index, row in cloze_df.iterrows():
  if row["label"] == 0: # correct would mean word2 is higher
    if row["word1_logit"] <	row["word2_logit"]: 
      predicted_labels.append(1)
    else: 
      predicted_labels.append(0)

  else: 
    if row["word1_logit"] >	row["word2_logit"]: 
      predicted_labels.append(1)
    else: 
      predicted_labels.append(0)

cloze_df["is_correct"] = predicted_labels

In [ ]:
# cloze_df["is_correct"].value_counts()
grp = cloze_df.groupby(["test_phrase", "word1", "word2", "is_correct"])["is_correct"].count()
pd.set_option("display.max_rows", grp.index.shape[0])

print(grp)

test_phrase          word1         word2        is_correct
 I am [MASK]!        exaggerating  blunt        0             14
                                                1             6 
                                   honest       0             10
                                                1             10
                                   serious      0             10
                                                1             10
                                   transparent  0             14
                                                1             6 
                                   unsarcastic  0             20
                     ironic        blunt        0             10
                                                1             10
                                   honest       0             10
                                                1             10
                                   serious      0             10
                               

## Create a function to do everything above automatically
gets batches

In [ ]:
import random
random.seed(10)

In [ ]:
# remove long parent comments and comments from the df so we don't run into memory issues later
print(df.shape)

df_shortsent = df[df['parent_comment'].str.split().str.len().lt(40)]
df_shortsent = df_shortsent[df_shortsent['comment'].str.split().str.len().lt(40)]

print(df_shortsent.shape)

(1010773, 10)
(855340, 10)


In [ ]:
def create_cloze(df):
                #  , num_examples):

  # s0 = df.label[df.label.eq(0)].sample(num_examples).index
  # s1 = df.label[df.label.eq(1)].sample(num_examples).index 
  # df_subset = df.loc[s0.union(s1)].reset_index()

  # comments = list(df_subset['comment'].values)
  # parent_comments = list(df_subset['parent_comment'].values)
  # labels = list(df_subset['label'].values)

  comments = list(df['comment'].values)
  parent_comments = list(df['parent_comment'].values)
  labels = list(df['label'].values)

  test_phrase1 = " I am [MASK]!"
  test_phrase2 = " I am being [MASK]!"
  test_phrase3 = " OP is [MASK]!"
  test_phrase4 = " OP is [MASK]."

  test_phrases = [test_phrase1, test_phrase2, test_phrase3, test_phrase4]

  word1 = ['joking', 'ironic', 'satirical', 'exaggerating', 'sarcastic', 'kidding']
  word2 = ['serious', 'honest', 'blunt', 'transparent', 'unsarcastic']

  pairs = list(itertools.product(word1, word2))

  df_parent_list = []
  df_comment_list = []
  df_label_list = []
  df_testphrase_list = []
  df_word1_list = []
  df_word2_list = []
  df_logit1_list = []
  df_logit2_list = []

  for phrase in test_phrases:
    for pair in pairs:
      comments_pet = [comment + phrase for comment in comments]
      combined = [list(i) for i in zip(parent_comments, comments_pet)]

      inputs = tokenizer(combined, padding=True, return_tensors="tf", max_length=40)
      mask_positions = np.where(inputs['input_ids'] == 103)
      out = model(inputs)
      pair_tokens = tokenizer.convert_tokens_to_ids(pair)

      for example_nr, (context, example) in enumerate(combined):
          df_parent_list.append(context)
          df_comment_list.append(example)
          df_label_list.append(labels[example_nr])
          df_testphrase_list.append(phrase)
          df_word1_list.append(pair[0])
          df_word2_list.append(pair[1])
          df_logit1_list.append(out[0][example_nr, mask_positions[1][example_nr]][pair_tokens[0]].numpy())
          df_logit2_list.append(out[0][example_nr, mask_positions[1][example_nr]][pair_tokens[1]].numpy())

  df_dict = {"parent_comment" : df_parent_list, 
              "comment" : df_comment_list, 
              "test_phrase" : df_testphrase_list, 
              "label" : df_label_list, 
              "word1" : df_word1_list, 
              "word2" : df_word2_list, 
              "word1_logit" : df_logit1_list, 
              "word2_logit" : df_logit2_list} 

  cloze_df = pd.DataFrame(df_dict)

  predicted_labels = []
  for index, row in cloze_df.iterrows():
    if row["label"] == 0: # correct would mean word2 is higher
      if row["word1_logit"] <	row["word2_logit"]: 
        predicted_labels.append(1)
      else: 
        predicted_labels.append(0)

    else: 
      if row["word1_logit"] >	row["word2_logit"]: 
        predicted_labels.append(1)
      else: 
        predicted_labels.append(0)

  cloze_df["is_correct"] = predicted_labels


  grp = cloze_df.groupby(["test_phrase", "word1", "word2", "is_correct"])["is_correct"].count()
  pd.set_option("display.max_rows", grp.index.shape[0])

  # print(grp)

  # return cloze_df, grp
  return (cloze_df, grp)



In [ ]:
# cloze_df_20, grp_20 = create_cloze(df, 10)
# cloze_df_50, grp_50 = create_cloze(df, 25)
# cloze_df_100, grp_100 = create_cloze(df, 50)
# cloze_df_200, grp_200 = create_cloze(df, 100)
# cloze_df_500, grp_500 = create_cloze(df, 250)
# cloze_df_1000, grp_1000 = create_cloze(df, 500)



In [ ]:
# most we'll have for a half is 500
# since we can't do a lot of examples at a time - let's run batches through the cloze function
s0 = df_shortsent.label[df_shortsent.label.eq(0)].sample(500).index
s1 = df_shortsent.label[df_shortsent.label.eq(1)].sample(500).index 

batches = {}
ind = 0
for x in range(0, 500, 50):
  print(ind)
  subset0 = s0[x : x+50]
  subset1 = s1[x : x+50]
  df_subset = df_shortsent.loc[subset0.union(subset1)].reset_index()
  (batch_df, batch_grp) = create_cloze(df_subset)
  name = "batch" + str(ind)
  batches[name] = (batch_df, batch_grp)
  ind += 1



# df_0 = df.loc[s0.union(s1)].reset_index()
# df_1 = df.loc[s0.union(s1)].reset_index()
# df_subset = df.loc[s0.union(s1)].reset_index()

0
1
2
3
4
5
6
7
8
9


In [ ]:
batches.keys()

dict_keys(['batch0', 'batch1', 'batch2', 'batch3', 'batch4', 'batch5', 'batch6', 'batch7', 'batch8', 'batch9'])

## LM for each set of examples
##Manually look at breakdown and decide pairs

### 50 examples per class

In [ ]:
data_50 = batches["batch0"][0]
data_50.groupby(["test_phrase", "word1", "word2", "is_correct"])["is_correct"].count()

test_phrase          word1         word2        is_correct
 I am [MASK]!        exaggerating  blunt        0             46 
                                                1             54 
                                   honest       0             51 
                                                1             49 
                                   serious      0             50 
                                                1             50 
                                   transparent  0             46 
                                                1             54 
                                   unsarcastic  0             100
                     ironic        blunt        0             50 
                                                1             50 
                                   honest       0             50 
                                                1             50 
                                   serious      0             50 
                 

In [ ]:
final_pairs50 = (('joking', 'transparent'), ('kidding', 'transparent'), ('joking', 'blunt'), ('ironic', 'transparent'), ('ironic', 'blunt'))

final_phrase50 = " I am being [MASK]!"

final_df_50 = pd.DataFrame()

temp_df = batches["batch0"][0]
filtered_batch_df_50 = temp_df[(temp_df['test_phrase'] == final_phrase) & (((temp_df['word1'] == "joking") & (temp_df['word2'] == "transparent")) | 
                                                                 ((temp_df['word1'] == "kidding") & (temp_df['word2'] == "transparent")) | 
                                                                 ((temp_df['word1'] == "joking") & (temp_df['word2'] == "blunt")) |
                                                                 ((temp_df['word1'] == "ironic") & (temp_df['word2'] == "transparent")) |
                                                                 ((temp_df['word1'] == "ironic") & (temp_df['word2'] == "blunt"))
                                                                 )]
  # print(filtered_batch_df.shape)                                                               
final_df_50 = final_df_50.append(filtered_batch_df_50)

print(cloze_df.shape)
print(final_df_50.shape)
final_df_50.head()

(2400, 9)
(500, 9)


,parent_comment,comment,test_phrase,label,word1,word2,word1_logit,word2_logit,is_correct
3200,"Hell just thinking that if you want to level a character from level 1, thats 97 levels you have to earn before you will even see your artifact.",It took 8 years for my characters to see their Artifact weapons. I am being [MASK]!,I am being [MASK]!,0,joking,blunt,5.980262,4.633221,0
3201,People are willing to pay $7.99 a month for one program?,Shit dude I pay 2.50 an episode to Google for shows I like I am being [MASK]!,I am being [MASK]!,0,joking,blunt,1.961727,2.252184,1
3202,member when I didn't give a fuck,Member when i downvoted your ass? I am being [MASK]!,I am being [MASK]!,1,joking,blunt,6.115184,6.269764,0
3203,"I feel like this might be cheating but, firearms instructor?",Yeah I know how you feel... I am being [MASK]!,I am being [MASK]!,0,joking,blunt,5.217716,5.733661,1
3204,Yahoo Email Surveillance: the Next Front in the Fight Against Mass Surveillance,I'm sure google and bing would never do such a thing at the government's behest either. I am being [MASK]!,I am being [MASK]!,1,joking,blunt,4.892037,4.933511,0


### 100 examples per class

In [185]:
data_100 = pd.DataFrame()
for b in list(batches.keys())[:2]:
  data_100 = data_100.append(batches[b][0])

data_100.groupby(["test_phrase", "word1", "word2", "is_correct"])["is_correct"].count()

# look for words/phrases where the is_correct 1 is > 100

test_phrase          word1         word2        is_correct
 I am [MASK]!        exaggerating  blunt        0             98 
                                                1             102
                                   honest       0             103
                                                1             97 
                                   serious      0             101
                                                1             99 
                                   transparent  0             96 
                                                1             104
                                   unsarcastic  0             200
                     ironic        blunt        0             98 
                                                1             102
                                   honest       0             100
                                                1             100
                                   serious      0             100
                 

In [ ]:
final_pairs100 = (('joking', 'transparent'), ('kidding', 'transparent'), ('joking', 'blunt'), ('ironic', 'transparent'), ('ironic', 'blunt'))

final_phrase100 = " I am being [MASK]!"

final_df_100 = pd.DataFrame()

for b in list(batches.keys())[:2]:
  temp_df = batches[b][0]
  filtered_batch_df_100 = temp_df[(temp_df['test_phrase'] == final_phrase) & (((temp_df['word1'] == "joking") & (temp_df['word2'] == "transparent")) | 
                                                                 ((temp_df['word1'] == "kidding") & (temp_df['word2'] == "transparent")) | 
                                                                 ((temp_df['word1'] == "joking") & (temp_df['word2'] == "blunt")) |
                                                                 ((temp_df['word1'] == "ironic") & (temp_df['word2'] == "transparent")) |
                                                                 ((temp_df['word1'] == "ironic") & (temp_df['word2'] == "blunt"))
                                                                 )]
  # print(filtered_batch_df.shape)                                                               
  final_df_100 = final_df_100.append(filtered_batch_df_100)

print(cloze_df.shape)
print(final_df_100.shape)
final_df_100.head()

(2400, 9)
(1000, 9)


,parent_comment,comment,test_phrase,label,word1,word2,word1_logit,word2_logit,is_correct
3200,I'm fair disappointed that I couldn't use this meme,Yeah because the Irish Presidency is such a powerful and important position I am being [MASK]!,I am being [MASK]!,1,joking,blunt,1.247952,1.797580,0
3201,"Well, I guess we can't ""brigade"" it and earn ourselves some two day bans. Thanks, Reddit!",Never understood the purpose of banning people from an anonymous site they can register an unverifiable account to within 30 seconds of their ban. I am being [MASK]!,I am being [MASK]!,0,joking,blunt,6.342561,6.009890,0
3202,And looked how that worked out,"Sorry, forgot the I am being [MASK]!",I am being [MASK]!,1,joking,blunt,5.774253,6.336384,0
3203,Lots of baking with my mum.,What kind of things do you guys bake? I am being [MASK]!,I am being [MASK]!,0,joking,blunt,6.223651,6.827527,1
3204,She hates DC.,Can't imagine why. I am being [MASK]!,I am being [MASK]!,1,joking,blunt,4.137773,5.107919,0


### 250 examples per class

In [187]:
data_250 = pd.DataFrame()
for b in list(batches.keys())[:5]:
  data_250 = data_250.append(batches[b][0])

data_250.groupby(["test_phrase", "word1", "word2", "is_correct"])["is_correct"].count()

# look for words/phrases where the is_correct 1 is > 250

test_phrase          word1         word2        is_correct
 I am [MASK]!        exaggerating  blunt        0             265
                                                1             235
                                   honest       0             252
                                                1             248
                                   serious      0             251
                                                1             249
                                   transparent  0             247
                                                1             253
                                   unsarcastic  0             500
                     ironic        blunt        0             250
                                                1             250
                                   honest       0             251
                                                1             249
                                   serious      0             250
                 

In [188]:
# final_pairs250 = (('joking', 'transparent'), ('kidding', 'transparent'), ('joking', 'blunt'), ('ironic', 'transparent'), ('ironic', 'blunt'))
final_pairs250 = (('joking', 'transparent'), ('kidding', 'transparent'), ('sarcastic', 'serious'), ('ironic', 'transparent'), ('joking', 'blunt'))

final_phrase250 = " I am being [MASK]!"

final_df_250 = pd.DataFrame()

for b in list(batches.keys())[:5]:
  temp_df = batches[b][0]
  filtered_batch_df_250 = temp_df[(temp_df['test_phrase'] == final_phrase) & (((temp_df['word1'] == "joking") & (temp_df['word2'] == "transparent")) | 
                                                                 ((temp_df['word1'] == "kidding") & (temp_df['word2'] == "transparent")) | 
                                                                 ((temp_df['word1'] == "joking") & (temp_df['word2'] == "blunt")) |
                                                                 ((temp_df['word1'] == "ironic") & (temp_df['word2'] == "transparent")) |
                                                                 ((temp_df['word1'] == "ironic") & (temp_df['word2'] == "blunt"))
                                                                 )]
  # print(filtered_batch_df.shape)                                                               
  final_df_250 = final_df_250.append(filtered_batch_df_250)

print(cloze_df.shape)
print(final_df_250.shape)
final_df_250.head()

(2400, 9)
(2500, 9)


,parent_comment,comment,test_phrase,label,word1,word2,word1_logit,word2_logit,is_correct
3200,I'm fair disappointed that I couldn't use this meme,Yeah because the Irish Presidency is such a powerful and important position I am being [MASK]!,I am being [MASK]!,1,joking,blunt,1.247952,1.797580,0
3201,"Well, I guess we can't ""brigade"" it and earn ourselves some two day bans. Thanks, Reddit!",Never understood the purpose of banning people from an anonymous site they can register an unverifiable account to within 30 seconds of their ban. I am being [MASK]!,I am being [MASK]!,0,joking,blunt,6.342561,6.009890,0
3202,And looked how that worked out,"Sorry, forgot the I am being [MASK]!",I am being [MASK]!,1,joking,blunt,5.774253,6.336384,0
3203,Lots of baking with my mum.,What kind of things do you guys bake? I am being [MASK]!,I am being [MASK]!,0,joking,blunt,6.223651,6.827527,1
3204,She hates DC.,Can't imagine why. I am being [MASK]!,I am being [MASK]!,1,joking,blunt,4.137773,5.107919,0


### 500 examples per class

In [189]:
data_500 = pd.DataFrame()
for b in list(batches.keys()):
  data_500 = data_500.append(batches[b][0])

data_500.groupby(["test_phrase", "word1", "word2", "is_correct"])["is_correct"].count()

# look for words/phrases where the is_correct 1 is > 500

test_phrase          word1         word2        is_correct
 I am [MASK]!        exaggerating  blunt        0             526 
                                                1             474 
                                   honest       0             497 
                                                1             503 
                                   serious      0             501 
                                                1             499 
                                   transparent  0             508 
                                                1             492 
                                   unsarcastic  0             1000
                     ironic        blunt        0             493 
                                                1             507 
                                   honest       0             499 
                                                1             501 
                                   serious      0             500 
   

In [190]:
# final_pairs500 = (('joking', 'transparent'), ('kidding', 'transparent'), ('joking', 'blunt'), ('ironic', 'transparent'), ('ironic', 'blunt'))

final_pairs500 = (('joking', 'transparent'), ('kidding', 'transparent'), ('joking', 'blunt'), ('sarcastic', 'serious'), ('kidding', 'blunt'))

final_phrase_500 = " I am being [MASK]!"



# run and get the logits dataframe with one phrase and the chosen pairs
# final_df = cloze_df[(cloze_df['test_phrase'] == final_phrase) & (((cloze_df['word1'] == "joking") & (cloze_df['word2'] == "transparent")) | 
#                                                                  ((cloze_df['word1'] == "kidding") & (cloze_df['word2'] == "transparent")) | 
#                                                                  ((cloze_df['word1'] == "joking") & (cloze_df['word2'] == "blunt")) |
#                                                                  ((cloze_df['word1'] == "ironic") & (cloze_df['word2'] == "transparent")) |
#                                                                  ((cloze_df['word1'] == "ironic") & (cloze_df['word2'] == "blunt"))
#                                                                  )]
final_df_500 = pd.DataFrame()
for b in batches.keys():
  temp_df = batches[b][0]
  filtered_batch_df_500 = temp_df[(temp_df['test_phrase'] == final_phrase) & (((temp_df['word1'] == "joking") & (temp_df['word2'] == "transparent")) | 
                                                                 ((temp_df['word1'] == "kidding") & (temp_df['word2'] == "transparent")) | 
                                                                 ((temp_df['word1'] == "joking") & (temp_df['word2'] == "blunt")) |
                                                                 ((temp_df['word1'] == "ironic") & (temp_df['word2'] == "transparent")) |
                                                                 ((temp_df['word1'] == "ironic") & (temp_df['word2'] == "blunt"))
                                                                 )]
  # print(filtered_batch_df.shape)                                                               
  final_df_500 = final_df_500.append(filtered_batch_df_500)

print(cloze_df.shape)
print(final_df_500.shape)
final_df_500.head()



(2400, 9)
(5000, 9)


,parent_comment,comment,test_phrase,label,word1,word2,word1_logit,word2_logit,is_correct
3200,I'm fair disappointed that I couldn't use this meme,Yeah because the Irish Presidency is such a powerful and important position I am being [MASK]!,I am being [MASK]!,1,joking,blunt,1.247952,1.797580,0
3201,"Well, I guess we can't ""brigade"" it and earn ourselves some two day bans. Thanks, Reddit!",Never understood the purpose of banning people from an anonymous site they can register an unverifiable account to within 30 seconds of their ban. I am being [MASK]!,I am being [MASK]!,0,joking,blunt,6.342561,6.009890,0
3202,And looked how that worked out,"Sorry, forgot the I am being [MASK]!",I am being [MASK]!,1,joking,blunt,5.774253,6.336384,0
3203,Lots of baking with my mum.,What kind of things do you guys bake? I am being [MASK]!,I am being [MASK]!,0,joking,blunt,6.223651,6.827527,1
3204,She hates DC.,Can't imagine why. I am being [MASK]!,I am being [MASK]!,1,joking,blunt,4.137773,5.107919,0


### Get Averages + Soft Labels

In [191]:
# for each comment - average the logits that are given to it (will get an average for 0 and average for 1 - whichever is higher is our soft label) - put this in a df

avg_log_50 = final_df_50.groupby(["parent_comment", "comment"])["word1_logit", "word2_logit", "label"].mean()
avg_log_100 = final_df_100.groupby(["parent_comment", "comment"])["word1_logit", "word2_logit", "label"].mean()
avg_log_250 = final_df_250.groupby(["parent_comment", "comment"])["word1_logit", "word2_logit", "label"].mean()
avg_log_500 = final_df_500.groupby(["parent_comment", "comment"])["word1_logit", "word2_logit", "label"].mean()


# avg_log

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [192]:
avg_log_50 = avg_log_50.reset_index(inplace=False)
avg_log_50 = avg_log_50.rename(columns={'word1_logit': 'word1_logit_avg', 'word2_logit': 'word2_logit_avg'})

avg_log_100 = avg_log_100.reset_index(inplace=False)
avg_log_100 = avg_log_100.rename(columns={'word1_logit': 'word1_logit_avg', 'word2_logit': 'word2_logit_avg'})

avg_log_250 = avg_log_250.reset_index(inplace=False)
avg_log_250 = avg_log_250.rename(columns={'word1_logit': 'word1_logit_avg', 'word2_logit': 'word2_logit_avg'})

avg_log_500 = avg_log_500.reset_index(inplace=False)
avg_log_500 = avg_log_500.rename(columns={'word1_logit': 'word1_logit_avg', 'word2_logit': 'word2_logit_avg'})

# avg_log

In [193]:
# find the soft label

soft_labels = []
for index, row in avg_log_50.iterrows():
  if row["word1_logit_avg"] <	row["word2_logit_avg"]: 
    soft_labels.append(0)
  else: 
    soft_labels.append(1)
avg_log_50["soft_labels"] = soft_labels

soft_labels = []
for index, row in avg_log_100.iterrows():
  if row["word1_logit_avg"] <	row["word2_logit_avg"]: 
    soft_labels.append(0)
  else: 
    soft_labels.append(1)
avg_log_100["soft_labels"] = soft_labels

soft_labels = []
for index, row in avg_log_250.iterrows():
  if row["word1_logit_avg"] <	row["word2_logit_avg"]: 
    soft_labels.append(0)
  else: 
    soft_labels.append(1)
avg_log_250["soft_labels"] = soft_labels

soft_labels = []
for index, row in avg_log_500.iterrows():
  if row["word1_logit_avg"] <	row["word2_logit_avg"]: 
    soft_labels.append(0)
  else: 
    soft_labels.append(1)
avg_log_500["soft_labels"] = soft_labels
# avg_log

In [194]:
# avg_log_500.head()

,parent_comment,comment,word1_logit_avg,word2_logit_avg,label,soft_labels
0,"""Are you just a nasty girl with negative behavior who ate up for no good but drama and trouble?"" (Image 3 and 4 should be switched)",what sort of fucked up self destructive sociopath uses linkedin for dating stuff? I am being [MASK]!,5.390528,6.044638,0,0
1,"""Bitcoin has brought a surge of new talent to the financial technology space.""",Wait a minute... There was financial technology before Bitcoin. I am being [MASK]!,5.443249,6.149345,1,0
2,"""Don't you like a man looking at your boobs all day?""","Mmm yeah I love men looking at my boobs all day, make's me feel like an object! I am being [MASK]!",4.477148,6.029073,1,0
3,"""Hey, do you know Tomb Raider?"" ""No""","""Oh great.. You're one of 'those' gamers, aren't you?"" I am being [MASK]!",6.566190,6.058030,1,1
4,"""Lightning follows the path of least resistance, which is why France suffers from the most lightning strikes.""","Cue the reminder that the French Resistance was one of, if not the largest one in WW2 and made D-Day much less bloody by cutting Nazi supply lines and providing intelligence to the Allies. I am being [MASK]!",5.308903,5.473442,0,0


In [196]:
# compare soft label to actual label - error analysis on this
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

print("50 labels per class")
# accuracy
acc = accuracy_score(avg_log_50["label"], avg_log_50["soft_labels"])
print('Accuracy: %.3f' % acc)

# precision
precision = precision_score(avg_log_50["label"], avg_log_50["soft_labels"], average='binary', zero_division = 0)
print('Precision: %.3f' % precision)

# recall
recall = recall_score(avg_log_50["label"], avg_log_50["soft_labels"], average='binary', zero_division = 0)
print('Recall: %.3f' % recall)

#f1
score = f1_score(avg_log_50["label"], avg_log_50["soft_labels"], average='binary')
print('F-Measure: %.3f' % score)

target_names = ['not sarc', 'sarc']
print(classification_report(avg_log_50["label"], avg_log_50["soft_labels"], target_names=target_names))



print("100 labels per class")
# accuracy
acc = accuracy_score(avg_log_100["label"], avg_log_100["soft_labels"])
print('Accuracy: %.3f' % acc)

# precision
precision = precision_score(avg_log_100["label"], avg_log_100["soft_labels"], average='binary', zero_division = 0)
print('Precision: %.3f' % precision)

# recall
recall = recall_score(avg_log_100["label"], avg_log_100["soft_labels"], average='binary', zero_division = 0)
print('Recall: %.3f' % recall)

#f1
score = f1_score(avg_log_100["label"], avg_log_100["soft_labels"], average='binary')
print('F-Measure: %.3f' % score)

target_names = ['not sarc', 'sarc']
print(classification_report(avg_log_100["label"], avg_log_100["soft_labels"], target_names=target_names))




print("250 labels per class")
# accuracy
acc = accuracy_score(avg_log_250["label"], avg_log_250["soft_labels"])
print('Accuracy: %.3f' % acc)

# precision
precision = precision_score(avg_log_250["label"], avg_log_250["soft_labels"], average='binary', zero_division = 0)
print('Precision: %.3f' % precision)

# recall
recall = recall_score(avg_log_250["label"], avg_log_250["soft_labels"], average='binary', zero_division = 0)
print('Recall: %.3f' % recall)

#f1
score = f1_score(avg_log_250["label"], avg_log_250["soft_labels"], average='binary')
print('F-Measure: %.3f' % score)

target_names = ['not sarc', 'sarc']
print(classification_report(avg_log_250["label"], avg_log_250["soft_labels"], target_names=target_names))



print("500 labels per class")
# accuracy
acc = accuracy_score(avg_log_500["label"], avg_log_500["soft_labels"])
print('Accuracy: %.3f' % acc)

# precision
precision = precision_score(avg_log_500["label"], avg_log_500["soft_labels"], average='binary', zero_division = 0)
print('Precision: %.3f' % precision)

# recall
recall = recall_score(avg_log_500["label"], avg_log_500["soft_labels"], average='binary', zero_division = 0)
print('Recall: %.3f' % recall)

#f1
score = f1_score(avg_log_500["label"], avg_log_500["soft_labels"], average='binary')
print('F-Measure: %.3f' % score)

target_names = ['not sarc', 'sarc']
print(classification_report(avg_log_500["label"], avg_log_500["soft_labels"], target_names=target_names))




50 labels per class
Accuracy: 0.530
Precision: 0.548
Recall: 0.340
F-Measure: 0.420
              precision    recall  f1-score   support

    not sarc       0.52      0.72      0.61        50
        sarc       0.55      0.34      0.42        50

    accuracy                           0.53       100
   macro avg       0.54      0.53      0.51       100
weighted avg       0.54      0.53      0.51       100

100 labels per class
Accuracy: 0.505
Precision: 0.507
Recall: 0.370
F-Measure: 0.428
              precision    recall  f1-score   support

    not sarc       0.50      0.64      0.56       100
        sarc       0.51      0.37      0.43       100

    accuracy                           0.51       200
   macro avg       0.51      0.51      0.50       200
weighted avg       0.51      0.51      0.50       200

250 labels per class
Accuracy: 0.496
Precision: 0.494
Recall: 0.312
F-Measure: 0.382
              precision    recall  f1-score   support

    not sarc       0.50      0.68    

## Bert with parent comment and comment

### Remove mask from comments

In [197]:
avg_log_50["comment"] = avg_log_50["comment"].str.replace('I am being [MASK]!', '', regex=False)
avg_log_100["comment"] = avg_log_100["comment"].str.replace('I am being [MASK]!', '', regex=False)
avg_log_250["comment"] = avg_log_250["comment"].str.replace('I am being [MASK]!', '', regex=False)
avg_log_500["comment"] = avg_log_500["comment"].str.replace('I am being [MASK]!', '', regex=False)


In [198]:
avg_log_500.head()


,parent_comment,comment,word1_logit_avg,word2_logit_avg,label,soft_labels
0,"""Are you just a nasty girl with negative behavior who ate up for no good but drama and trouble?"" (Image 3 and 4 should be switched)",what sort of fucked up self destructive sociopath uses linkedin for dating stuff?,5.390528,6.044638,0,0
1,"""Bitcoin has brought a surge of new talent to the financial technology space.""",Wait a minute... There was financial technology before Bitcoin.,5.443249,6.149345,1,0
2,"""Don't you like a man looking at your boobs all day?""","Mmm yeah I love men looking at my boobs all day, make's me feel like an object!",4.477148,6.029073,1,0
3,"""Hey, do you know Tomb Raider?"" ""No""","""Oh great.. You're one of 'those' gamers, aren't you?""",6.566190,6.058030,1,1
4,"""Lightning follows the path of least resistance, which is why France suffers from the most lightning strikes.""","Cue the reminder that the French Resistance was one of, if not the largest one in WW2 and made D-Day much less bloody by cutting Nazi supply lines and providing intelligence to the Allies.",5.308903,5.473442,0,0


### Run bert using the soft labels - 50

In [199]:
parent_comments = avg_log_50.parent_comment.values
comments = avg_log_50.comment.values
labels = avg_log_50.soft_labels.values

In [200]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Loading BERT tokenizer...


loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6


In [201]:
combined = [list(i) for i in zip(parent_comments, comments)]

In [202]:
combined

[['"Are you just a nasty girl with negative behavior who ate up for no good but drama and trouble?" (Image 3 and 4 should be switched)',
  'what sort of fucked up self destructive sociopath uses linkedin for dating stuff? '],
 ['"Women in tech" ? &amp;#3232;\\_&amp;#3232;',
  'Well, at least we don\'t have "Men in Beauty Industry". '],
 ["90% of those mods have been added in the last 12-months. But that's probably just a coincidence.",
  "No, it's a right-wing conspiracy! "],
 ['ASDERHNIP ADRIAN SHEPHARD',
  "Hey hey, that's just fans trolling themselves! "],
 ["Also, there's nothing responsible about drinking Old Milwaukee...",
  'Financial responsibility. '],
 ["Alternatives to TrueCrypt? I setup my cousin with TrueCrypt when he left for China 2 years ago. Now that TrueCrypt isn't available what would my sysadmin bros and gals recommend?",
  "I've heard of this thing called cryptolocker, works flawlessly. "],
 ['And another one. Holy shit this one got 440 likes.',
  'Clearly is has a

In [203]:
batch_encoded_dict = tokenizer.batch_encode_plus(
                      combined,
                      add_special_tokens = True,
                      max_length = 50,
                      pad_to_max_length = True,
                      return_token_type_ids=True,
                      return_attention_mask = True,
                      return_tensors = 'tf'
                     )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [204]:
labels = tf.convert_to_tensor(labels)

In [205]:
labels

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0])>

In [206]:
def example_to_features(input_ids, attention_masks, token_type_ids, y):
  return {"input_ids": input_ids,
          "token_type_ids": token_type_ids,
          "attention_mask": attention_masks}, y

dataset = tf.data.Dataset.from_tensor_slices((batch_encoded_dict['input_ids'], batch_encoded_dict['token_type_ids'], batch_encoded_dict['attention_mask'], 
                                              labels)).map(example_to_features)

In [207]:
# Train on 50% of the dataset, Validate on 25% of the dataset, Test on 25% of the dataset
# Calculate the number of samples to include in each set
dataset_size = len(dataset)
train_size = int(0.5 * dataset_size)
val_size = int(0.25 * dataset_size)
test_size = int(0.25 * dataset_size)
dataset = dataset.shuffle(dataset_size)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = dataset.skip(test_size)
test_dataset = dataset.take(test_size)

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(test_size))

   50 training samples
   25 validation samples
   25 test samples


In [208]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.5, num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.5,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



In [209]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = labels.flatten()
    acc = np.sum(preds_flat == labels_flat) / len(labels_flat)
    return {'accuracy': acc}

In [210]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=5e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Tensorflow: setting up strategy
loading weights file https://huggingface.co/bert-base-cased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/01800f4158e284e2447020e0124bc3f6aea3ac49848e744594f7cce8ee5ac0a4.a7137b2090d9302d722735af604b4c142ec9d1bfc31be7cbbe230aea9d5cfb76.h5
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are instantiating a Trainer but W&B is not

In [211]:
import transformers

transformers.logging.set_verbosity_info()

In [212]:
trainer.train()

***** Running training *****
  Num examples = 50
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 4
  Total optimization steps = 8


Training took: 0:00:12.660386


In [213]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 25
  Num examples in used in evaluation = 64
  Batch size = 64


{'eval_loss': 0.6800963282585144, 'eval_accuracy': 0.640625, 'epoch': 2.0, 'step': 8}


{'eval_accuracy': 0.640625, 'eval_loss': 0.6800963282585144}

In [214]:
tf.keras.backend.clear_session()

### Run bert using the soft labels - 100

In [215]:
parent_comments = avg_log_100.parent_comment.values
comments = avg_log_100.comment.values
labels = avg_log_100.soft_labels.values

In [216]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [217]:
combined = [list(i) for i in zip(parent_comments, comments)]

In [218]:
# combined

In [219]:
batch_encoded_dict = tokenizer.batch_encode_plus(
                      combined,
                      add_special_tokens = True,
                      max_length = 50,
                      pad_to_max_length = True,
                      return_token_type_ids=True,
                      return_attention_mask = True,
                      return_tensors = 'tf'
                     )

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [220]:
labels = tf.convert_to_tensor(labels)

In [221]:
labels

<tf.Tensor: shape=(200,), dtype=int64, numpy=
array([0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0])>

In [222]:
def example_to_features(input_ids, attention_masks, token_type_ids, y):
  return {"input_ids": input_ids,
          "token_type_ids": token_type_ids,
          "attention_mask": attention_masks}, y

dataset = tf.data.Dataset.from_tensor_slices((batch_encoded_dict['input_ids'], batch_encoded_dict['token_type_ids'], batch_encoded_dict['attention_mask'], 
                                              labels)).map(example_to_features)

In [223]:
# Train on 50% of the dataset, Validate on 25% of the dataset, Test on 25% of the dataset
# Calculate the number of samples to include in each set
dataset_size = len(dataset)
train_size = int(0.5 * dataset_size)
val_size = int(0.25 * dataset_size)
test_size = int(0.25 * dataset_size)
dataset = dataset.shuffle(dataset_size)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = dataset.skip(test_size)
test_dataset = dataset.take(test_size)

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(test_size))

  100 training samples
   50 validation samples
   50 test samples


In [224]:
# from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.5, num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.5,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



In [225]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = labels.flatten()
    acc = np.sum(preds_flat == labels_flat) / len(labels_flat)
    return {'accuracy': acc}

In [226]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=5e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Tensorflow: setting up strategy
loading weights file https://huggingface.co/bert-base-cased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/01800f4158e284e2447020e0124bc3f6aea3ac49848e744594f7cce8ee5ac0a4.a7137b2090d9302d722735af604b4c142ec9d1bfc31be7cbbe230aea9d5cfb76.h5
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are instantiating a Trainer but W&B is not

In [227]:
# import transformers

transformers.logging.set_verbosity_info()

In [228]:
trainer.train()

***** Running training *****
  Num examples = 100
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 7
  Total optimization steps = 14


Training took: 0:00:13.234917


In [229]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 50
  Num examples in used in evaluation = 64
  Batch size = 64


{'eval_loss': 0.7347804307937622, 'eval_accuracy': 0.515625, 'epoch': 2.0, 'step': 14}


{'eval_accuracy': 0.515625, 'eval_loss': 0.7347804307937622}

In [230]:
tf.keras.backend.clear_session()

### Run bert using the soft labels - 250

In [231]:
parent_comments = avg_log_250.parent_comment.values
comments = avg_log_250.comment.values
labels = avg_log_250.soft_labels.values

In [232]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6


In [233]:
combined = [list(i) for i in zip(parent_comments, comments)]

In [234]:
batch_encoded_dict = tokenizer.batch_encode_plus(
                      combined,
                      add_special_tokens = True,
                      max_length = 50,
                      pad_to_max_length = True,
                      return_token_type_ids=True,
                      return_attention_mask = True,
                      return_tensors = 'tf'
                     )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [235]:
labels = tf.convert_to_tensor(labels)

In [236]:
labels

<tf.Tensor: shape=(500,), dtype=int64, numpy=
array([0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
     

In [237]:
def example_to_features(input_ids, attention_masks, token_type_ids, y):
  return {"input_ids": input_ids,
          "token_type_ids": token_type_ids,
          "attention_mask": attention_masks}, y

dataset = tf.data.Dataset.from_tensor_slices((batch_encoded_dict['input_ids'], batch_encoded_dict['token_type_ids'], batch_encoded_dict['attention_mask'], 
                                              labels)).map(example_to_features)

In [238]:
# Train on 50% of the dataset, Validate on 25% of the dataset, Test on 25% of the dataset
# Calculate the number of samples to include in each set
dataset_size = len(dataset)
train_size = int(0.5 * dataset_size)
val_size = int(0.25 * dataset_size)
test_size = int(0.25 * dataset_size)
dataset = dataset.shuffle(dataset_size)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = dataset.skip(test_size)
test_dataset = dataset.take(test_size)

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(test_size))

  250 training samples
  125 validation samples
  125 test samples


In [239]:
config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.5, num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.5,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



In [240]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = labels.flatten()
    acc = np.sum(preds_flat == labels_flat) / len(labels_flat)
    return {'accuracy': acc}

In [241]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=5e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Tensorflow: setting up strategy
loading weights file https://huggingface.co/bert-base-cased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/01800f4158e284e2447020e0124bc3f6aea3ac49848e744594f7cce8ee5ac0a4.a7137b2090d9302d722735af604b4c142ec9d1bfc31be7cbbe230aea9d5cfb76.h5
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are instantiating a Trainer but W&B is not

In [242]:
transformers.logging.set_verbosity_info()

In [243]:
trainer.train()

***** Running training *****
  Num examples = 250
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 16
  Total optimization steps = 32


Training took: 0:00:15.662529


In [244]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 125
  Num examples in used in evaluation = 128
  Batch size = 64


{'eval_loss': 0.681489884853363, 'eval_accuracy': 0.625, 'epoch': 2.0, 'step': 32}


{'eval_accuracy': 0.625, 'eval_loss': 0.681489884853363}

In [245]:
tf.keras.backend.clear_session()

### Run bert using the soft labels - 500

In [246]:
parent_comments = avg_log_500.parent_comment.values
comments = avg_log_500.comment.values
labels = avg_log_500.soft_labels.values

In [247]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6


In [248]:
combined = [list(i) for i in zip(parent_comments, comments)]

In [249]:
batch_encoded_dict = tokenizer.batch_encode_plus(
                      combined,
                      add_special_tokens = True,
                      max_length = 50,
                      pad_to_max_length = True,
                      return_token_type_ids=True,
                      return_attention_mask = True,
                      return_tensors = 'tf'
                     )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [250]:
labels = tf.convert_to_tensor(labels)

In [251]:
labels

<tf.Tensor: shape=(1000,), dtype=int64, numpy=
array([0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
    

In [252]:
def example_to_features(input_ids, attention_masks, token_type_ids, y):
  return {"input_ids": input_ids,
          "token_type_ids": token_type_ids,
          "attention_mask": attention_masks}, y

dataset = tf.data.Dataset.from_tensor_slices((batch_encoded_dict['input_ids'], batch_encoded_dict['token_type_ids'], batch_encoded_dict['attention_mask'], 
                                              labels)).map(example_to_features)

In [253]:
# Train on 50% of the dataset, Validate on 25% of the dataset, Test on 25% of the dataset
# Calculate the number of samples to include in each set
dataset_size = len(dataset)
train_size = int(0.5 * dataset_size)
val_size = int(0.25 * dataset_size)
test_size = int(0.25 * dataset_size)
dataset = dataset.shuffle(dataset_size)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = dataset.skip(test_size)
test_dataset = dataset.take(test_size)

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(test_size))

  500 training samples
  250 validation samples
  250 test samples


In [254]:
config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.5, num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.5,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



In [255]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = labels.flatten()
    acc = np.sum(preds_flat == labels_flat) / len(labels_flat)
    return {'accuracy': acc}

In [256]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=5e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Tensorflow: setting up strategy
loading weights file https://huggingface.co/bert-base-cased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/01800f4158e284e2447020e0124bc3f6aea3ac49848e744594f7cce8ee5ac0a4.a7137b2090d9302d722735af604b4c142ec9d1bfc31be7cbbe230aea9d5cfb76.h5
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are instantiating a Trainer but W&B is not

In [257]:
transformers.logging.set_verbosity_info()

In [258]:
trainer.train()

***** Running training *****
  Num examples = 500
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 32
  Total optimization steps = 64


Training took: 0:00:19.229913


In [259]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 250
  Num examples in used in evaluation = 256
  Batch size = 64


{'eval_loss': 0.5999228954315186, 'eval_accuracy': 0.7265625, 'epoch': 2.0, 'step': 64}


{'eval_accuracy': 0.7265625, 'eval_loss': 0.5999228954315186}

In [260]:
tf.keras.backend.clear_session()

## Error Analysis

### Get predictions for 500 examples per class (1000 total)

In [263]:
pred_labels_1000 = trainer.predict(test_dataset).label_ids

***** Running Prediction *****
  Num examples in dataset = 250
  Batch size = 64


In [264]:
pred_labels_1000

array([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 0])

In [269]:
test_np = list(test_dataset.as_numpy_iterator())
true_labels_1000 = []
for elem in test_np:
  true_labels_1000.append(elem[1])

In [271]:
# accuracy
acc = accuracy_score(true_labels_1000, pred_labels_1000)
print('Accuracy: %.3f' % acc)

# precision
precision = precision_score(true_labels_1000, pred_labels_1000, average='binary', zero_division = 0)
print('Precision: %.3f' % precision)

# recall
recall = recall_score(true_labels_1000, pred_labels_1000, average='binary', zero_division = 0)
print('Recall: %.3f' % recall)

#f1
score = f1_score(true_labels_1000, pred_labels_1000, average='binary')
print('F-Measure: %.3f' % score)

target_names = ['not sarc', 'sarc']
print(classification_report(true_labels_1000, pred_labels_1000, target_names=target_names))


Accuracy: 0.568
Precision: 0.256
Recall: 0.309
F-Measure: 0.280
              precision    recall  f1-score   support

    not sarc       0.72      0.66      0.69       182
        sarc       0.26      0.31      0.28        68

    accuracy                           0.57       250
   macro avg       0.49      0.49      0.49       250
weighted avg       0.59      0.57      0.58       250



### Specific Examples

#### Untokenize the comments and put them into a list

In [281]:
untokenized_1000 = []
for elem in test_np:
  elem_input_ids = elem[0]["input_ids"]
  elem_tokens = tokenizer.convert_ids_to_tokens(elem_input_ids)
  untokenized_1000.append(" ".join(elem_tokens))

In [283]:
untokenized_1000[:5]

['[CLS] A Different Kind of Up ##grade ( Power ##line adapt ##ers for Wire ##d Internet ) [SEP] what exactly are these ? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 "[CLS] * Ha ##zed ? * You miss ##pelled indicted . [SEP] Probably not if it wasn ' t his intent to la ##und ##er . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]",
 '[CLS] Damn . I loved Cloud Atlas and would love to see a 4 hour version . That film had a lot of flaws , but it was so ambitious the flaws felt ne ##gli ##gible . [SEP] Nah man I want another re ##boo ##t [SEP] [PAD] [PAD] [PAD]',
 '[CLS] Just like they ruined the Elder Sc ##rolls series , the GT ##A series , the Fall ##out series , the Witch ##er series . . [SEP] I hated how Counter ##st ##rik ##e and Team fortress ruined my half ##life and q ##u

In [286]:
fp = []
fn = []
tp = []
tn = []

for ind, (l1, l2) in enumerate(zip(true_labels_1000, pred_labels_1000)):
  if l1 == 0 and l2 == 1: #predicted as sarcastic but actually not
    fp.append(ind)
  elif l1 == 1 and l2 == 0: # predicted as not but actually are sarcastic
    fn.append(ind)
  elif l1 == 1 and l2 == 1: # predicted as sarc and correct
    tp.append(ind)
  else: # predicted as not and it's correct
    tn.append(ind)

#### Examples that were predicted as sarcastic but are actually not sarcastic

In [289]:
[untokenized_1000[i] for i in fp]

["[CLS] * Ha ##zed ? * You miss ##pelled indicted . [SEP] Probably not if it wasn ' t his intent to la ##und ##er . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]",
 '[CLS] People are willing to pay $ 7 . 99 a month for one program ? [SEP] Shit dude I pay 2 . 50 an episode to Google for shows I like [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] Exactly this has nothing to do with tax ##payers . Just a somewhat dubious but very successful company . [SEP] One that " dubious ##ly " and " allegedly " might possibly be engaging in things like front - running and other similar ass ##orted * ah ##em [SEP]',
 '[CLS] Going to Kris ##py K ##rem ##e when the " Hot " sign is on . . . [SEP] They have an app to let you know when the hot ##light is on by the way . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

#### Examples that were predicted as not sarcastic but are actually sarcastic

In [290]:
[untokenized_1000[i] for i in fn]

['[CLS] Damn . I loved Cloud Atlas and would love to see a 4 hour version . That film had a lot of flaws , but it was so ambitious the flaws felt ne ##gli ##gible . [SEP] Nah man I want another re ##boo ##t [SEP] [PAD] [PAD] [PAD]',
 '[CLS] Just like they ruined the Elder Sc ##rolls series , the GT ##A series , the Fall ##out series , the Witch ##er series . . [SEP] I hated how Counter ##st ##rik ##e and Team fortress ruined my half ##life and q ##ua ##ke ##world gameplay ! [SEP]',
 "[CLS] You ' re going to spa ##m 4 ##chan with gay p ##orn . How , exactly , will anyone be able to notice ? [SEP] Holy shit they ' ve been raiding / b / for years ! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]",
 "[CLS] Let ' s form a cult [SEP] I ' ll be the mascot . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]",
 '[CLS] What wa

#### Examples that were predicted as sarcastic correctly

In [291]:
[untokenized_1000[i] for i in tp]

["[CLS] Guess you didn ' t watch N ##XT Brooklyn . [SEP] Sorry , I forgot something [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]",
 '[CLS] does that look like Syria to you , O ##P ? Do these guys look like IDF ? ! [SEP] There are songs about the sprawling green hills of Syria . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 "[CLS] It ' s odd considering the bullshit he dealt with when it comes to the \\ # Can ##cel ##C ##ol ##bert fi ##asco with Sue ##y Park . [SEP] Yeah but see that was just one lone crazy [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]",
 '[CLS] Typical Tu ##mb ##l ##rina [SEP] U ##gh , this is so in ##con ##side ##rate for people like me who is a black person trapped inside a white person body who is trapped inside an as ##ian body . [SEP] [

#### Examples that were predicted as not sarcastic correctly

In [292]:
[untokenized_1000[i] for i in tn]

['[CLS] A Different Kind of Up ##grade ( Power ##line adapt ##ers for Wire ##d Internet ) [SEP] what exactly are these ? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 "[CLS] E ##L ##I ##5 : How does a group like the Il ##lum ##ina ##ti , who were originally a group of free think [SEP] With the amount of deleted comments in this thread I feel like we ' re only being shown what the Il ##lum ##ina [SEP]",
 '[CLS] He might be grown in a physical sense but clearly not a mental one [SEP] Big man , mental mid ##get [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 "[CLS] I ' ll take the spot up on that balcony to the side , that way I don ' t have to pretend to rush over to help . [SEP] F ##Y ##I , that ##s the city hall balcony : ) [SEP] [PAD] [PAD] [PAD] 